### 1. Checklist

###### Checklist for logs: 

|job ID|log| reason acc. to elasticsearch | reason acc to script| |
|-|----------|----------|----------|-----|
| baf77c21-bdd5-0aa2-8731-000000000385|9ef7ce17ae108085e8a26e1f2046e1cf73e8c976.json|Infrastructure - Matlab License |Infrastructure - Matlab License |y|
| 86623622-6e74-9368-74a8-000000000028|882dafcc748b4f0695ce486241a05ad2.json|?| Test(s) failed|y|
|16911dd9-d58b-df18-7329-000000000ba3|e850514f59cb3334754f2641fc63f20f46af56e7.json|FuSa Violation found|Main Category: FuSa Violation Found (Check 'bazel_wrapper_log.txt' on the logs folder), Subcategory: , Pattern: Failed actions detected in bazel_wrapper_log.txt file|y|
|No ID provided||?|redirected output - Fusa Small Checks - Check-Bazel-rules-in-FuSa-critical-components, Subcategory: Fusa Small Checks - Check-Bazel-rules-in-FuSa-critical-components|?|
|No ID provided||?| redirected output - default, Subcategory: default|?|
|c65a4cd1-edf7-790d-cd51-000000000042|0752ac127d21a70af1765ae29afc53d978224f25.json|python|Test(s) failed, Subcategory: |?|
|No ID provided||python|Main Category: python, Subcategory: ,|y||
|b6d2a4bc-700f-7546-f838-000000000042|8e89be688f5b331654a6f44975a7c6c23dd3b71c.json|Compiler (gnu)  | Bazel - error executing command, Subcategory: error executing command|X|
||00e0b3d030f7fd0eef0c4b016e3d5e7e2b51a69f.json|Bazel (failed on target analysis)|Not flagged / No match found|X|
|No ID provided|b819bae8dfb1923b8f9dc3c1b4bb2eed8c3fa2a3.json|Compiler (clang)|Bazel - error executing command, Subcategory: error executing command|X|
|NO ID provided|cd5c0bbd5821d9d31f41442f78bcae86221e65a0.json|Bazel(missing input file)|Bazel - missing input file, Subcategory: missing input file, Pattern: ERROR:(.*)input file\(s\) do not exist |?|
|NO ID provided||Bazel(missing input file)|Bazel - missing input file, Subcategory: missing input file, Pattern: ERROR:(.*)missing input file '.*' |?|

Why same file gets "unknown error" & a match? -->  Because 18 logs but 34 errors - Needs Dealing with
- 005fc583d4745e2470010c57b6422bcbc5c5fb0e271a7f11d72248e702e1090a (2 times)
- 0067a8788a1a5e30cecc3c159a8f5b6546a46cd5fb18253c82de30e8d1e7a988 (4 times)
- 0077401c9568efb6267a2c6ffd1e696ffe70e479d7c5e69413d3d7aea3105035 (4 times)
- e850514f59cb3334754f2641fc63f20f46af56e7 (6 times)
- 0079e5bcee8a7318e88e6d40be15c242ad522e2af6cbc1cdbeecf4beb0e743c7 (3 times)
- 0752ac127d21a70af1765ae29afc53d978224f25 (4 times)
- 8e89be688f5b331654a6f44975a7c6c23dd3b71c (3 times)
- 9ef7ce17ae108085e8a26e1f2046e1cf73e8c976 (2 times)
- cd5c0bbd5821d9d31f41442f78bcae86221e65a0 (2 times)

- fix inconsistencies in pattern? 
- fix script for all cases ( decision!) --> use existing _find_errors ()

### 2. Code

In [1]:
import json
import extract_build_failures.error_patterns as error_patterns
import re
import os
import textwrap

INFRA_PATTERNS = error_patterns.INFRA_PATTERNS
BUILD_PATTERNS = error_patterns.BUILD_PATTERNS

# compile patterns from error_patterns.py
def compile_patterns(patterns_dict):
    compiled_patterns = {}
    for main_category, patterns in patterns_dict.items():
        if isinstance(patterns, dict):
            for sub_category, sub_patterns in patterns.items():
                for pattern in sub_patterns:
                    compiled_patterns.setdefault((main_category, sub_category), []).append(re.compile(pattern))
        else:
            for pattern in patterns:
                compiled_patterns.setdefault((main_category, ""), []).append(re.compile(pattern))
    return compiled_patterns


# check log entry against patterns
def check_log_entry(log_entry, compiled_patterns):
    matches = []
    for (main_category, sub_category), regex_list in compiled_patterns.items():
        for regex in regex_list:
            if regex.search(log_entry):
                category = f"{main_category} - {sub_category}" if sub_category else main_category
                matches.append((category, sub_category, regex.pattern))
    return matches

# process files
def process_log_files(directory_path, compiled_patterns):
    summary = {
        'tasks_summary': {},
        'msg_summary': {}
    }
    for filename in filter(lambda f: f.endswith('.json'), os.listdir(directory_path)):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            log_entries = json.load(file)
            for log in log_entries:
                task_id = log.get('id', 'No ID provided')
                name = log.get('name', 'No task name provided')
                msg = log['msg']
                task_key = f"{name} (ID: {task_id})"
                summary['tasks_summary'].setdefault(task_key, 0)
                summary['tasks_summary'][task_key] += 1
                summary['msg_summary'].setdefault(msg, 0)
                summary['msg_summary'][msg] += 1

                stdout_text = "\n".join(log.get('stdout_lines', []))
                matches = check_log_entry(stdout_text, compiled_patterns)
                for match in matches:
                    main_category, sub_category, pattern = match
                    print(f"Current File: {file_path}")
                    print(task_key)
                    print(f"Main Category: {main_category}, Subcategory: {sub_category}, Pattern: {pattern}")
    return summary

# format summary for readability
def format_summary_to_screen_width(summary, terminal_width=150):
    formatted_summary = ""
    for key, value in summary.items():
        if isinstance(value, dict):
            formatted_summary += f"{key}:\n"
            for sub_key, sub_value in value.items():
                wrapped_sub_value = textwrap.fill(str(sub_value), terminal_width - 4)
                formatted_summary += f"  {sub_key}: {wrapped_sub_value}\n"
        else:
            wrapped_value = textwrap.fill(str(value), terminal_width)
            formatted_summary += f"{key}: {wrapped_value}\n"
        formatted_summary += "-" * terminal_width + "\n"
    return formatted_summary

# Usage:
compiled_infra_patterns = compile_patterns(INFRA_PATTERNS)
compiled_build_patterns = compile_patterns(BUILD_PATTERNS)
directory_path = 'preprocessed_logs'
all_compiled_patterns = {**compiled_infra_patterns, **compiled_build_patterns}
logs_directory_path = 'logs'
output_directory_path = 'preprocessed_logs'

summary = process_log_files(directory_path, all_compiled_patterns)
formatted_summary = format_summary_to_screen_width(summary)
print(formatted_summary)

Current File: preprocessed_logs\005fc583d4745e2470010c57b6422bcbc5c5fb0e271a7f11d72248e702e1090a__job-output_cropped.json
docker-run (ID: 2aa698b8-b2fd-a900-8da6-00000000002d)
Main Category: Test(s) failed, Subcategory: , Pattern: \/\/.*\s+FAILED in [0-9.]+s
Current File: preprocessed_logs\00609ed82bfdb509cf935a6101a636b891e25ef1a08268849b236609f42650d3__job-output_cropped.json
bazel-execute (ID: 620460c8-edcb-fb02-f8d4-000000000333)
Main Category: Test(s) failed, Subcategory: , Pattern: \/\/.*\s+FAILED in [0-9.]+s
Current File: preprocessed_logs\0063c906e5290ca5c11bc7c8efba154fc8dcbcb03643bcda0f54f793bd9af5b5__job-output_cropped.json
docker-run (ID: ea446e25-7667-a36d-6c98-000000000028)
Main Category: Test(s) failed, Subcategory: , Pattern: \/\/.*\s+FAILED in [0-9.]+s
Current File: preprocessed_logs\0065bc74ca0b24d30a11b96a73f1fb8588a624962b67bec41fcde6a862c5acb9__job-output_cropped.json
docker-exec (ID: aa463074-4aee-806c-398d-0000000000cf)
Main Category: Bazel - error executing comm